# Line Plots

## 2D line plots

This example plots a sine wave, cosine wave, and ricker wavelet and demonstrates how Graphic Features can be modified by slicing!

In [4]:
import numpy as np
import fastplotlib as fpl
from sidecar import Sidecar

### First generate some data.

In [39]:
# linspace, create 100 evenly spaced x values from -10 to 10
xs = np.linspace(-10, 10, 100)
# sine wave
ys = np.sin(xs)
sine = np.dstack([xs, ys])[0]

# cosine wave
ys = np.cos(xs) + 5
cosine = np.dstack([xs, ys])[0]

# sinc function
a = 0.5
ys = np.sinc(xs) * 3 + 8
sinc = np.dstack([xs, ys])[0]

### We will plot all of it on the same plot. Each line plot will be an individual Graphic, you can have any combination of graphics on a plot.

In [40]:
# Create a plot instance
plot_l = fpl.Plot()

# plot sine wave, use a single color
sine_graphic = plot_l.add_line(data=sine, thickness=5, colors="magenta")

# you can also use colormaps for lines!
cosine_graphic = plot_l.add_line(data=cosine, thickness=12, cmap="autumn")

# or a list of colors for each datapoint
colors = ["r"] * 25 + ["purple"] * 25 + ["y"] * 25 + ["b"] * 25
sinc_graphic = plot_l.add_line(data=sinc, thickness=5, colors = colors)

sc = Sidecar(title="lines")

with sc: 
    display(plot_l.show())

RFBOutputContext()

### "stretching" the camera, useful for large timeseries data

Set `maintain_aspect = False` on a camera, and then use the right mouse button and move the mouse to stretch and squeeze the view!

You can also click the `1:1` button to toggle this.

In [7]:
plot_l.camera.maintain_aspect = False

### reset the plot area

In [8]:
plot_l.auto_scale(maintain_aspect=True)

### Graphic features support slicing! :D

In [9]:
# indexing of colors
cosine_graphic.colors[:15] = "magenta"
cosine_graphic.colors[90:] = "red"
cosine_graphic.colors[60] = "w"

# indexing to assign colormaps to entire lines or segments
sinc_graphic.cmap[10:50] = "gray"
sine_graphic.cmap = "seismic"

# more complex indexing, set the blue value directly from an array
cosine_graphic.colors[65:90, 0] = np.linspace(0, 1, 90-65)

### You can capture changes to a graphic feature as events

In [10]:
def callback_func(event_data):
    print(event_data)

# Will print event data when the color changes
cosine_graphic.colors.add_event_handler(callback_func)

In [11]:
# more complex indexing of colors
# from point 15 - 30, set every 3rd point as "cyan"
cosine_graphic.colors[15:50:3] = "cyan"

FeatureEvent @ 0x7f4ab642b410
type: colors
pick_info: {'index': range(15, 50, 3), 'collection-index': None, 'world_object': <weakproxy at 0x7f4ab5f5be20 to Line at 0x7f4ab6509e90>, 'new_data': array([[0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.],
       [0., 1., 1., 1.]], dtype=float32)}



### Graphic data is itself also indexable

In [12]:
cosine_graphic.data[10:50:5, :2] = sine[10:50:5]
cosine_graphic.data[90:, 1] = 7
cosine_graphic.data[0] = np.array([[-10, 0, 0]])

/home/clewis7/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:31: UserWarning: converting int64 array to int32
  warn(f"converting {array.dtype} array to int32")


### Toggle the presence of a graphic within the scene

In [13]:
sinc_graphic.present = False

In [14]:
sinc_graphic.present = True

### You can create callbacks to this too, for example to re-scale the plot w.r.t. graphics that are present in the scene


In [15]:
sinc_graphic.present.add_event_handler(plot_l.auto_scale)

sinc_graphic.present = False

In [16]:
sinc_graphic.present = True

### You can set the z-positions of graphics to have them appear under other graphics

In [17]:
img = np.random.rand(20, 100)

plot_l.add_image(img, name="image", cmap="gray")

# z axix position -1 so it is below all the lines
plot_l["image"].position_z = -1
plot_l["image"].position_x = -50

In [18]:
sc.close()

### Can also have fancy indexing of colormaps

In [22]:
plot = fpl.Plot()

plot.add_line(sine, thickness=10, name="sine")

sc = Sidecar(title="line plot")

with sc:
    display(plot.show())

RFBOutputContext()

In [24]:
plot["sine"].cmap = "jet"

In [25]:
plot["sine"].cmap.values = sine[:, 1]

In [26]:
plot["sine"].cmap.values = cosine[:, 1]

In [27]:
plot["sine"].cmap = "viridis"

In [28]:
cmap_values = [0] * 25 + [1] * 5 + [2] * 50 + [3] * 20
plot["sine"].cmap.values = cmap_values

In [47]:
sc.close()

## Line Collections

In [62]:
plot = fpl.Plot()

plot.add_line_collection([sine, cosine, sinc], thickness=5, cmap="viridis")

sc = Sidecar(title="line collection")

with sc:
    display(plot.show())

RFBOutputContext()

In [63]:
sc.close()

## Line Stack

In [68]:
plot = fpl.Plot()

plot.add_line_stack([sine, cosine, sinc], thickness=5, separation=1, cmap="viridis")

sc = Sidecar(title="line stack")

with sc:
    display(plot.show())

RFBOutputContext()